In [55]:
import pandas as pd
import numpy as np
import pickle


In [68]:
dataset = pd.read_csv("./Datasets/playlist-sample-ds2.csv")

In [70]:
pd.set_option('display.max_colwidth', 0)
columns = ['name', 'duration_ms', 'album_name', 'artist_name', 'track_name']
keepColumn = 'track_name'
columns.remove(keepColumn)
df2 = dataset.drop(columns, axis=1)
tracks = df2[keepColumn].unique()
playlists = df2['pid'].unique()
df2 = df2.groupby("pid")[keepColumn].apply(lambda x: list(x))
df2 = df2.to_frame()[keepColumn]
df2

pid
365       [Breezeblocks, Clouds & Cream, In The Aeroplane Over The Sea, Hold On, Lisztomania, Only For You, October, Love You Madly, Trojans, Undercover Martyn, Ain't No Rest for the Wicked - Original Version, Gimme All Your Love, If I ever feel better, Why'd You Only Call Me When You're High?, White Lies, Cruel, Down By The River, Shake Me Down, 1901, Do You, Ultimate Painting, Fluorescent Adolescent, Ain't Horrible, Sleepwalkin', Take a Walk, What You Know, Parade, Animal, Green Is The Colour - 2011 Remastered Version, Chartreuse, Last Nite, Too Late To Say Goodbye, After the Disco, Step, The Less I Know The Better, I Can't Handle Change, In A Feeling Like This, Baby Blue - 2010 - Remaster, Tighten Up, Trouble, Don't Wanna Fight, Stronger Than That, Flaws, If I Were A Portal, Go It Alone, Thirsty Man, Cigarette Daydreams - Live From Guitar Center, Things Happen, Heart Is Black, Evil Forces, Mountain At My Gates]                                                                     

In [71]:
from collections import defaultdict
ohe_df = defaultdict(dict)
for row in df2.keys():
    for music in df2[row]:
        if (len(ohe_df[music]) == 0):
            ohe_df[music] = []
        ohe_df[music].append(row)
# ohe_df['Breezeblocks']
playlists_musics_df = defaultdict(dict)

for p in playlists:
    for t in tracks:
        if (len(playlists_musics_df[t]) == 0):
            playlists_musics_df[t] = []
        if p in ohe_df[t]:
            playlists_musics_df[t].append(1)
        else:
            playlists_musics_df[t].append(0)

In [72]:
freq_df = pd.DataFrame.from_dict(playlists_musics_df, orient='index', columns=playlists).transpose()
freq_df

,Breezeblocks,Clouds & Cream,In The Aeroplane Over The Sea,Hold On,Lisztomania,Only For You,October,Love You Madly,Trojans,Undercover Martyn,...,West End Girls - 2001 Remastered Version,It's A Sin - 2001 Remastered Version,What Have I Done To Deserve This? (With Dusty Springfield) - 2001 Remastered Version,Always On My Mind,Where The Streets Have No Name - I Can't Take My Eyes Off You; 2003 Remastered Version,Enola Gay - 2003 - Remaster,Souvenir - 2003 Digital Remaster,Joan Of Arc (Maid Of Orleans) - 2003 Digital Remaster,Tesla Girls,Locomotion
365,1,1,1,1,1,1,1,1,1,1,...,0,0,0,0,0,0,0,0,0,0
516,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
539,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
765,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
100331,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
98280,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
98749,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
98871,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
98881,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [73]:
from mlxtend.preprocessing import TransactionEncoder
from mlxtend.frequent_patterns import apriori, association_rules

freqItems = apriori(freq_df, min_support=0.01, use_colnames=True, verbose=1)
freqItems.sort_values('support')

/home/daniel/Documents/UFMG/CloudComputing/tp2-cloud-computing/venv/lib/python3.10/site-packages/mlxtend/frequent_patterns/fpcommon.py:110: DeprecationWarning: DataFrames with non-bool types result in worse computationalperformance and their support might be discontinued in the future.Please use a DataFrame with bool type
  warnings.warn(


Processing 36 combinations | Sampling itemset size 4 3 2


,support,itemsets
475,0.010,"(Joy to the World, It's Beginning To Look A Lot Like Christmas)"
476,0.010,"(Blue Christmas, All I Want for Christmas Is You)"
478,0.010,"(All I Want for Christmas Is You, It's Beginning To Look A Lot Like Christmas)"
479,0.010,"(Heroes (we could be), Blame)"
481,0.010,"(Bounce Back, Bad and Boujee (feat. Lil Uzi Vert))"
...,...,...
108,0.030,(Skinny Love)
157,0.032,(You)
417,0.032,(HUMBLE.)
60,0.034,(Intro)


In [74]:
rules = association_rules(freqItems, metric="confidence", min_threshold=0.01)
#rules = rules[rules['support'] >= 0.015]
with open('./trained_model.pkl', 'wb') as file:
    pickle.dump(rules, file)